In [13]:
import notebookimport

ds= __import__("Report - Driver Splits")

In [22]:
import sqlite3
from IPython.display import HTML
import pandas as pd

dbname='mexico19.db'
conn = sqlite3.connect(dbname)
rally='Mexico'
rc='RC1'
year=2019

In [19]:
q_drivers='SELECT `driver.code` AS code  FROM startlists st JOIN startlist_classes sc ON sc.entryid = st.entryid AND name="{}"'

q_classes='SELECT DISTINCT(name) AS name FROM startlists st JOIN startlist_classes sc ON sc.entryid = st.entryid'

#Need to look this up properly
q_stages= ['SS{}'.format(i) for i in range(1,17) ]

In [20]:
def stage_chart(rc,driver, stage):
    s2 = ds.getDriverSplitsReport(conn, rally, stage, driver, rc)
    display(HTML(s2))

In [23]:
import ipywidgets as widgets
from ipywidgets import interact

classes = widgets.Dropdown(
    options=pd.read_sql(q_classes,conn)['name'].to_list(),
    value='RC1', description='Class:', disabled=False )

drivers = widgets.Dropdown(
    options=pd.read_sql(q_drivers.format(classes.value),conn)['code'].to_list(),
    description='Driver:', disabled=False)

stages = widgets.Dropdown(
    options=q_stages, description='Stage:', disabled=False)

def update_drivers(*args):
    qdrivers = q_drivers.format(classes.value)
    driverlist = pd.read_sql(qdrivers,conn)['code'].to_list()
    print(driverlist)
    drivers.options = driverlist
    
classes.observe(update_drivers, 'value')

interact(stage_chart, rc=classes, driver=drivers, stage=stages);

interactive(children=(Dropdown(description='Class:', options=('RC1', 'RC2', 'NAT'), value='RC1'), Dropdown(des…

## How About More Control?

What about we let the user decide what to allow in each chart?

In [8]:
classes2 = widgets.Dropdown(
    options=pd.read_sql(q_classes,conn)['name'].to_list(),
    value='RC1',
    description='Class:',
    disabled=False,
)

drivers2 = widgets.Dropdown(
    options=pd.read_sql(q_drivers.format(classes2.value),conn)['code'].to_list(),
    #value='NEU',
    description='Driver:',
    disabled=False,
)

stages2 = widgets.Dropdown(
    options=q_stages, description='Stage:', disabled=False)


def stage_chart2(rc,driver, stage, bars, roadPos, waypointRank):
    dropcols=[]
    if not roadPos:
        dropcols.append('Road Position')
    if not waypointRank:
        dropcols.append('Waypoint Rank')
        
    s2 = ds.getDriverSplitsReport(conn, rally, stage, driver, rc,
                                  bars=bars, dropcols=dropcols)
    display(HTML(s2))

splitBars = widgets.Checkbox( value=True, description='Split bars:',
                           disabled=False )
    
roadPos = widgets.Checkbox( value=True, description='Road pos:',
                           disabled=False )

waypointRank = widgets.Checkbox( value=True, description='Waypoint Rank:',
                           disabled=False )

def update_drivers2(*args):
    qdrivers = q_drivers.format(classes2.value)
    driverlist = pd.read_sql(qdrivers,conn)['code'].to_list()
    print(driverlist)
    drivers2.options = driverlist
    
classes2.observe(update_drivers2, 'value')


interact(stage_chart2, rc=classes2, driver=drivers2,
         stage=stages2, bars=splitBars, roadPos=roadPos,
        waypointRank=waypointRank);

interactive(children=(Dropdown(description='Class:', options=('RC1', 'RC2', 'RC4', 'NAT4'), value='RC1'), Drop…

## HTML Table to PNG

There is a jquery plugin for this - could we use this in the widget app to donwload the table as png?

- https://stackoverflow.com/questions/38425931/download-table-as-png-using-jquery/40644383#40644383
- https://tableexport.v5.travismclarke.com/#tableexport
- https://html2canvas.hertzen.com/ / https://github.com/niklasvh/html2canvas
- https://w3lessons.info/export-html-table-to-excel-csv-json-pdf-png-using-jquery/ export to pdf
- https://github.com/simonbengtsson/jsPDF-AutoTable